In [ ]:
from Modules import *
import datetime
import time

def download_with_date_range(ticker, start_date, end_date, interval="1d"):
    """
    Download stock data with specific date range
    
    Parameters:
    ticker (str): Stock ticker symbol
    start_date (str): Start date in format 'YYYY-MM-DD'
    end_date (str): End date in format 'YYYY-MM-DD'
    interval (str): Data interval
    
    Returns:
    pandas.DataFrame: Stock data
    """
    try:
        stock = yf.Ticker(ticker)
        data = stock.history(start=start_date, end=end_date, interval=interval)
        data = data.reset_index()
        data = data.drop(columns=['Dividends','Stock Splits'],axis=1)
        return data
    except Exception as e:
        print(f"Error downloading data: {e}")
        return None
    

def wwma(values, n):
    """
     J. Welles Wilder's EMA 
    """
    return values.ewm(alpha=1/n, adjust=False).mean()

def atr(data, n=20):
    # data = df.copy()
    high = data["High"]
    low = data["Low"]
    close = data["Close"]
    data['tr0'] = abs(high - low)
    data['tr1'] = abs(high - close.shift())
    data['tr2'] = abs(low - close.shift())
    tr = data[['tr0', 'tr1', 'tr2']].max(axis=1)
    atr = wwma(tr, n)
    data['atr'] = atr
    
    data = data.drop(columns=['tr0','tr1','tr2'], axis=1)
    
    return data

KeyboardInterrupt: 

Filter stocks

In [ ]:
from finvizfinance.screener.overview import Overview
foverview = Overview()
filters_dict = {'Market Cap.':'+Mid (over $2bln)', 'Average Volume' : 'Over 500K'}
foverview.set_filter(filters_dict=filters_dict)
df = foverview.screener_view()
df.head()

,Ticker,Company,Sector,Industry,Country,Market Cap,P/E,Price,Change,Volume
0,A,Agilent Technologies Inc,Healthcare,Diagnostics & Research,USA,4.135000e+10,34.20,145.85,-0.0050,1470057.0
1,AA,Alcoa Corp,Basic Materials,Aluminum,USA,1.018000e+10,8.95,39.31,-0.0148,6057081.0
2,AAL,American Airlines Group Inc,Industrials,Airlines,USA,8.390000e+09,15.28,12.71,-0.0522,53514473.0
3,AAOI,Applied Optoelectronics Inc,Technology,Communication Equipment,USA,2.300000e+09,NaN,36.87,-0.0094,5298958.0
4,AAON,AAON Inc,Industrials,Building Products & Equipment,USA,8.430000e+09,70.45,103.34,-0.0155,594991.0


Create list of Stocks

In [ ]:
listOfStocks = list(np.array(df['Ticker']))
print(listOfStocks)
print("Number of Stocks: ",str(len(listOfStocks))) 

['A', 'AA', 'AAL', 'AAOI', 'AAON', 'AAP', 'AAPL', 'ABBV', 'ABEV', 'ABM', 'ABNB', 'ABR', 'ABT', 'ABTC', 'ABVX', 'ACAD', 'ACGL', 'ACHC', 'ACHR', 'ACI', 'ACIW', 'ACLS', 'ACLX', 'ACM', 'ACMR', 'ACN', 'ADBE', 'ADC', 'ADI', 'ADM', 'ADMA', 'ADP', 'ADPT', 'ADSK', 'ADT', 'AEE', 'AEG', 'AEM', 'AEO', 'AEP', 'AER', 'AES', 'AFL', 'AFRM', 'AG', 'AGCO', 'AGI', 'AGIO', 'AGNC', 'AHL', 'AHR', 'AI', 'AIG', 'AJG', 'AKAM', 'AKR', 'AKRO', 'AL', 'ALAB', 'ALB', 'ALC', 'ALE', 'ALGM', 'ALGN', 'ALH', 'ALHC', 'ALK', 'ALKS', 'ALKT', 'ALL', 'ALLE', 'ALLY', 'ALNY', 'ALSN', 'ALV', 'AM', 'AMAT', 'AMBA', 'AMBP', 'AMCR', 'AMD', 'AME', 'AMGN', 'AMH', 'AMKR', 'AMRC', 'AMRX', 'AMRZ', 'AMSC', 'AMT', 'AMTM', 'AMX', 'AMZN', 'ANET', 'ANF', 'AON', 'AOS', 'APA', 'APD', 'APG', 'APGE', 'APH', 'APLD', 'APLE', 'APLS', 'APO', 'APP', 'APPN', 'APTV', 'AQN', 'AR', 'ARCC', 'ARE', 'ARES', 'ARLO', 'ARM', 'ARMK', 'AROC', 'ARQT', 'ARW', 'ARWR', 'ARX', 'AS', 'ASAN', 'ASB', 'ASH', 'ASML', 'ASO', 'ASTS', 'ASX', 'ATAT', 'ATEC', 'ATI', 'ATKR', 'A

Import historic data    

In [ ]:
print("Number of Stocks: ",str(len(listOfStocks))) 

for z in range(len(listOfStocks)):
    if(z<10 | z%50==0):
        print(z)
    try:
        data = download_with_date_range(listOfStocks[z],'1990-01-01','2025-12-31','1d')
        atr(data)
        data['ema8'] = data['Close'].ewm(span=8, adjust=False).mean()
        data['ema21'] = data['Close'].ewm(span=21, adjust=False).mean()
        data['ema34'] = data['Close'].ewm(span=34, adjust=False).mean()
        data['ema55'] = data['Close'].ewm(span=55, adjust=False).mean()
        data['ema89'] = data['Close'].ewm(span=89, adjust=False).mean()
        data['sma50'] = data['Close'].rolling(window=50).mean()
        data['sma100'] = data['Close'].rolling(window=100).mean()
        data['sma200'] = data['Close'].rolling(window=200).mean()
        data = data.drop(columns=['tr0','tr1','tr2'],axis=1)

        countUpTrend = []
        countDownTrend = []

        ema8 = np.array(data['ema8'])
        ema21 = np.array(data['ema21'])
        ema34 = np.array(data['ema34'])
        ema55 = np.array(data['ema55'])
        ema89 = np.array(data['ema89'])

        sma50 = np.array(data['sma50'])
        sma100 = np.array(data['sma100'])
        sma200 = np.array(data['sma200'])

        close = np.array(data['Close'])

        cumUpTrend = []
        cumDownTrend = []

        for i in range(len(data)):
            countUpTrend.append(0)
            countDownTrend.append(0)
            cumUpTrend.append(0)
            cumDownTrend.append(0)
            if((ema8[i] > ema21[i] > ema34[i] > ema55[i] > ema89[i] > sma200[i]) & (close[i] > sma200[i])):
                countUpTrend[i] = 1
            else:
                countUpTrend[i] = 0
            if((ema8[i] < ema21[i] < ema34[i] < ema55[i] < ema89[i] < sma200[i]) & (close[i] < sma200[i])):
                countDownTrend[i] = 1
            else:
                countDownTrend[i] = 0
            
            if((countUpTrend[i-1] == 1) & (countUpTrend[i] == 1)):
                cumUpTrend[i] = cumUpTrend[i-1]+1
            else:
                cumUpTrend[i] = 0
            
            if((countDownTrend[i-1] == 1) & (countDownTrend[i] == 1)):
                cumDownTrend[i] = cumDownTrend[i-1]+1
            else:
                cumDownTrend[i] = 0

        data['cumUpTrend'] = cumUpTrend
        data['cumDownTrend'] = cumDownTrend

        import pandas_ta as ta

        adx = ta.adx(data["High"], data["Low"], data["Close"], length=13)
        data['adx'] = list(np.array(adx['ADX_13']))


        get_RSI(data, "Close", 2)

        close = np.array(data['Close'])
        high = np.array(data['High'])
        low = np.array(data['Low'])
        x = ta.stoch(data['High'], data['Low'], data['Close'],8, 3, 3)
        data['K'] = x['STOCHk_8_3_3']
        data['D'] = x['STOCHd_8_3_3']
        data.dropna(inplace=True)
        # display(data.tail(3))

        upTrend = data['cumUpTrend'].iloc[-1]
        downTrend = data['cumDownTrend'].iloc[-1]
        adx = data['adx'].iloc[-1]
        rsi_yest = data['RSI'].iloc[-2]
        rsi_today = data['RSI'].iloc[-1]
        slowD = data['D'].iloc[-1]


        if((upTrend >= 40) & (adx >= 20) & (rsi_yest <= 10) & (rsi_today>10) & (slowD <= 40)):
            display(listOfStocks[z])
        elif((downTrend >= 20) & (adx >= 20) & (rsi_yest >= 90) & (rsi_today<90) & (slowD >= 60)):
            display(listOfStocks[z])
        else:
            ""

    except:
        ""

Number of Stocks:  1627


$BLDR: possibly delisted; no price data found  (1d 1990-01-01 -> 2025-12-31)


Error downloading data: "['Dividends', 'Stock Splits'] not found in axis"


$CHKP: possibly delisted; no price data found  (1d 1990-01-01 -> 2025-12-31)


Error downloading data: "['Dividends', 'Stock Splits'] not found in axis"


...

'KD'

'KSPI'

'LTH'

'LYB'

'NWL'